In [4]:
# imports
import osmnx as ox
import nest_asyncio
nest_asyncio.apply()

from streetscape import GridBuilder, ObsBuilder, GSVRetriever

## Street Dataset

In [5]:
osm_street = ox.graph_from_place('Ann Arbor, MI')

In [6]:
g = GridBuilder()
streets = g.graph2gdf(osm_street)

In [7]:
print(len(streets))
streets.head()

57269


,osmid,name,length,geometry
0,8722810,Walter Drive,10.036,"LINESTRING (-83.7804416 42.288292, -83.7805635..."
1,444795290,NaN,6.683,"LINESTRING (-83.7804416 42.288292, -83.7804411..."
2,444795290,NaN,9.641,"LINESTRING (-83.7804416 42.288292, -83.7804424..."
3,8722810,Walter Drive,110.892,"LINESTRING (-83.7804416 42.288292, -83.7803952..."
4,42390861,NaN,25.832,"LINESTRING (-83.756935 42.3090174, -83.7569972..."


## Create candidiate grid points
create a 1D grid along with street segments

In [5]:
grids = g.create(streets)

53037 grid points created


In [6]:
grids.head()

,geometry,point_id,seg_end,segment_id,metadata_url
0,POINT (-0.07336670000000001 50.9952681),0,1,5199427,https://maps.googleapis.com/maps/api/streetvie...
1,POINT (-0.07338906852798188 50.99528232036349),1,0,5199427,https://maps.googleapis.com/maps/api/streetvie...
2,POINT (-0.07349814150221824 50.99535106056079),2,0,5199427,https://maps.googleapis.com/maps/api/streetvie...
3,POINT (-0.07361968674006909 50.99539422770838),3,0,5199427,https://maps.googleapis.com/maps/api/streetvie...
4,POINT (-0.07374123197791994 50.99543739485598),4,0,5199427,https://maps.googleapis.com/maps/api/streetvie...


## Find GSV points
From each grid candidiate point, we will send a query asking the location of gsv points.

In [7]:
%%time
# only first 10 grids
gsv_gdf = ObsBuilder().create(grids.head(10))

6 observation points retrieved.
CPU times: user 63.5 ms, sys: 7.71 ms, total: 71.2 ms
Wall time: 406 ms


In [8]:
gsv_gdf.head()

,index,copyright,date,pano_id,geometry
0,0,© Google,2017-06,7VbmzX28HsayLzftqCxUKg,POINT (-0.0733641 50.9952888)
1,2,© Google,2017-06,tCcpSq6Dh5-jwreEtLNy3w,POINT (-0.07350600352815902 50.99534625974742)
2,3,© Google,2017-06,Ag-jya4mCO2cFPNPd0VArA,POINT (-0.0736048 50.9953789)
3,4,© Google,2017-06,hBw7nyFYbNEmgqnP9a2bMQ,POINT (-0.07375464036488383 50.99542532553686)
4,5,© Google,2017-06,pYKClTXjyQXYrHGKUB87lA,POINT (-0.07388683471449316 50.99546628328553)


## Retrieve GSV images

Although google's cbk API provides an option to download the whole panorama, 
it is not an officially released API. This means we are not sure utilizing the cbk API 
is within the terms of use. 
If you are interested in using cbk API, see robolyst [streetview package](https://github.com/robolyst/streetview)

Here, we are going to use the official street view api even though it does not allow reconstruct the whole
panorama. We are going to use piecewise streetviews from each point.

The api format is:
https://maps.googleapis.com/maps/api/streetview?pano={}&size={}x{}&heading={}&fov={}&key={}&source=outdoor

In [10]:
gsv_urls = GSVRetriever().create_urls(gsv_gdf, npics=4)
print(len(gsv_urls))
gsv_urls.head()

24


,gsv_name,gsv_url,pano_id
0,image_0_0.jpg,https://maps.googleapis.com/maps/api/streetvie...,7VbmzX28HsayLzftqCxUKg
1,image_0_90.jpg,https://maps.googleapis.com/maps/api/streetvie...,7VbmzX28HsayLzftqCxUKg
2,image_0_180.jpg,https://maps.googleapis.com/maps/api/streetvie...,7VbmzX28HsayLzftqCxUKg
3,image_0_270.jpg,https://maps.googleapis.com/maps/api/streetvie...,7VbmzX28HsayLzftqCxUKg
4,image_2_0.jpg,https://maps.googleapis.com/maps/api/streetvie...,tCcpSq6Dh5-jwreEtLNy3w


In [12]:
GSVRetriever().retrieve_images(gsv_urls, "images")